In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('../data/processed/processed.csv')

In [3]:
df.shape

(3439, 9)

In [4]:
df.sample(3)

,kms_driven,owner,location,mileage,power,price,brand,engine,age
172,48700.0,first,pune,53.0,19.0,72000.0,Bajaj,NaN,6
3051,26200.0,second,mumbai,53.0,14.0,35000.0,Hero,150.0,6
1302,15000.0,first,mumbai,63.0,14.0,77200.0,Suzuki,150.0,2


In [5]:
X = df.drop('price',axis=1)
y = df['price']

In [6]:
X.head()

,kms_driven,owner,location,mileage,power,brand,engine,age
0,5947.0,first,other,53.0,19.0,Bajaj,NaN,4
1,11000.0,first,delhi,40.0,19.8,Royal Enfield,350.0,7
2,13568.0,first,delhi,63.0,14.0,Suzuki,150.0,5
3,20000.0,first,other,40.0,19.8,Royal Enfield,350.0,3
4,10143.0,first,delhi,55.0,8.0,Hero,NaN,3


In [7]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=.25,random_state=11)

In [8]:
# y = y.apply(np.log1p)

## Category Encoding

In [9]:
X.head()

,kms_driven,owner,location,mileage,power,brand,engine,age
0,5947.0,first,other,53.0,19.0,Bajaj,NaN,4
1,11000.0,first,delhi,40.0,19.8,Royal Enfield,350.0,7
2,13568.0,first,delhi,63.0,14.0,Suzuki,150.0,5
3,20000.0,first,other,40.0,19.8,Royal Enfield,350.0,3
4,10143.0,first,delhi,55.0,8.0,Hero,NaN,3


## Impute missing values

In [10]:
from sklearn.impute import KNNImputer

num_cols = [ col for col in X.columns if X[col].dtypes != 'object' ]
print(num_cols)

['kms_driven', 'mileage', 'power', 'engine', 'age']


In [11]:
imputer = KNNImputer(n_neighbors=7,weights='distance')

imputer.fit(X_train[num_cols])

# X_train[num_cols] = imputer.transform(X_train[num_cols])
# X_test[num_cols] = imputer.transform(X_test[num_cols])

X_train.loc[:][num_cols] = imputer.fit_transform(X_train[num_cols])
X_test.loc[:][num_cols] = imputer.transform(X_test[num_cols])

In [12]:
print('Missing Values in Train set',X_train.isnull().sum())
print('Missing Values in Test set',X_test.isnull().sum())

Missing Values in Train set kms_driven    0
owner         0
location      0
mileage       0
power         0
brand         0
engine        0
age           0
dtype: int64
Missing Values in Test set kms_driven    0
owner         0
location      0
mileage       0
power         0
brand         0
engine        0
age           0
dtype: int64


In [13]:
X_train.head()

,kms_driven,owner,location,mileage,power,brand,engine,age
2407,17000.0,first,other,40.0,19.00,Bajaj,220.000000,4
336,94000.0,third,delhi,38.5,20.11,Bajaj,204.289882,1
2768,4900.0,first,mumbai,50.0,14.30,Bajaj,199.194491,5
1129,15000.0,first,delhi,55.0,9.30,Hero,110.000000,5
2964,10009.0,first,other,35.0,25.00,UM,240.797939,3


In [14]:
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder,MinMaxScaler

category_transformer = ColumnTransformer([
    ("kms_driven_engine_min_max_scaler",MinMaxScaler(),[0,6,3,4]),
    ("owner_ordinal_enc",OrdinalEncoder(categories=[['fourth','third','second','first']],handle_unknown='ignore',dtype=np.int16),[1]),
    ("brand_location_ohe",OneHotEncoder(sparse=False,handle_unknown='error',drop='first',),[2,5]),
],remainder='passthrough')



## Model Building

In [15]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

from sklearn import set_config

set_config(display='diagram')

In [16]:

def build_pipeline_with_estimator(estimator):
    return Pipeline([
    ('category_transformer',category_transformer),
    ('estimator',estimator),
])


In [17]:
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error
from sklearn.model_selection import cross_val_score
# MAPE
def mape(targets, predictions):
    return np.mean(np.abs((targets - predictions)) / targets) * 100

# Adjusted R^2
def adj_r2(ind_vars, targets, predictions):
    r2 = r2_score(targets, predictions)
    n = ind_vars.shape[0]
    k = ind_vars.shape[1]
    return 1-((1-r2)*(n-1)/(n-k-1))

# Model performance check
def model_perf(model, inp, out,cross_val=True):

    y_pred = model.predict(inp)
    y_act = out.values

    cross_val_ = cross_val_score(model, inp,out,cv=10).mean() if cross_val else None
    

    return pd.DataFrame({
                "RMSE": np.sqrt(mean_squared_error(y_act, y_pred)),
                "MAE": mean_absolute_error(y_act, y_pred),
                "MAPE": mape(y_act, y_pred),
                "R^2": r2_score(y_act, y_pred),
                "Adjusted R^2": adj_r2(inp, y_act, y_pred),
                "Cross Val Score (Mean)": cross_val_ if cross_val else None
           }, index=[0])

## LinearRegression

In [18]:
linear_regressor = build_pipeline_with_estimator(LinearRegression())

linear_regressor.fit(X_train,y_train)

print('Linear Regression Train Performance.\n')
print(model_perf(linear_regressor,X_train,y_train,True))

print('Linear Regression Test Performance.\n')
print(model_perf(linear_regressor,X_test,y_test,True))

Linear Regression Train Performance.

           RMSE           MAE       MAPE       R^2  Adjusted R^2  \
0  16988.565913  12482.367334  18.865792  0.817687      0.817119   

   Cross Val Score (Mean)  
0                0.812923  
Linear Regression Test Performance.

           RMSE           MAE       MAPE       R^2  Adjusted R^2  \
0  17989.187845  13054.918696  19.997175  0.799415       0.79753   

   Cross Val Score (Mean)  
0                0.787729  


## RandomForest

In [19]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV

model = build_pipeline_with_estimator(RandomForestRegressor())

params = {
    'estimator__criterion': ['mse','mae'],
    'estimator__n_estimators': [100,110,120,130],
    'estimator__max_depth': [5,10,15,20,25,30],
    'estimator__min_samples_split': range(2,20),
    'estimator__max_features': ['auto','sqrt','log2'],
}

model = RandomizedSearchCV(model,params,cv=5)

model.fit(X_train,y_train)

print('RandomForest Train Performance.\n')
print(model_perf(model,X_train,y_train,False))

print('RandomForest Test Performance.\n')
print(model_perf(model,X_test,y_test,False))

RandomForest Train Performance.

          RMSE          MAE      MAPE       R^2  Adjusted R^2  \
0  9153.505655  6652.491563  9.335059  0.947073      0.946908   

  Cross Val Score (Mean)  
0                   None  
RandomForest Test Performance.

           RMSE           MAE       MAPE       R^2  Adjusted R^2  \
0  14935.895729  10895.729282  15.524417  0.861727      0.860427   

  Cross Val Score (Mean)  
0                   None  


## XGBoost

In [20]:
from xgboost import XGBRegressor

xgboost = build_pipeline_with_estimator(XGBRegressor())

xgboost.fit(X_train,y_train)

print('xgboost Train Performance.\n')
print(model_perf(xgboost,X_train,y_train))

print('xgboost Test Performance.\n')
print(model_perf(xgboost,X_test,y_test))

xgboost Train Performance.

          RMSE          MAE      MAPE       R^2  Adjusted R^2  \
0  7030.686406  4805.135542  6.569586  0.968775      0.968678   

   Cross Val Score (Mean)  
0                0.853562  
xgboost Test Performance.

           RMSE           MAE       MAPE       R^2  Adjusted R^2  \
0  14549.876103  10510.877809  14.781715  0.868782      0.867549   

   Cross Val Score (Mean)  
0                0.804053  


## KNeighborsRegressor

In [21]:
from sklearn.neighbors import KNeighborsRegressor

model = build_pipeline_with_estimator(KNeighborsRegressor())

params = {
    'estimator__n_neighbors': [3,5,7,9],
    'estimator__algorithm': ['ball_tree','kd_tree','brute'],
    'estimator__leaf_size': [5,10,30,32,35]
}

model = GridSearchCV(model,params,cv=5)

model.fit(X_train,y_train)

print('KNeighborsRegressor Train Performance.\n')
print(model_perf(model,X_train,y_train))

print('KNeighborsRegressor Test Performance.\n')
print(model_perf(model,X_test,y_test))

KNeighborsRegressor Train Performance.

           RMSE          MAE       MAPE       R^2  Adjusted R^2  \
0  13767.189839  9829.483676  13.935315  0.880272      0.879899   

   Cross Val Score (Mean)  
0                0.817754  
KNeighborsRegressor Test Performance.

           RMSE           MAE       MAPE       R^2  Adjusted R^2  \
0  16564.118191  11991.214419  17.440206  0.829937      0.828338   

   Cross Val Score (Mean)  
0                0.785274  


## GradientBoostingRegressor

In [22]:
from sklearn.ensemble import GradientBoostingRegressor

model = build_pipeline_with_estimator(GradientBoostingRegressor())

# params = {
#     'estimator__loss': ['ls','lad','huber','quantile'],
#     'estimator__learning_rate': [0.1,0.01,0.001],
#     'estimator__n_estimators': [100],
#     'estimator__criterion': ['friedman_mse','mse'],
# }

# model = RandomizedSearchCV(model,params,cv=5)

model.fit(X_train,y_train)

print('GradientBoostingRegressor Train Performance.\n')
print(model_perf(model,X_train,y_train))

print('GradientBoostingRegressor Test Performance.\n')
print(model_perf(model,X_test,y_test))

GradientBoostingRegressor Train Performance.

          RMSE          MAE      MAPE       R^2  Adjusted R^2  \
0  13323.67338  9777.050485  13.96837  0.887862      0.887513   

   Cross Val Score (Mean)  
0                0.863695  
GradientBoostingRegressor Test Performance.

           RMSE           MAE       MAPE       R^2  Adjusted R^2  \
0  14609.920389  10510.369966  14.960252  0.867697      0.866453   

   Cross Val Score (Mean)  
0                0.842533  
